In [ ]:
# !pip install pandas
# !pip install numpy 
# !pip install scikit-learn


In [1]:
import pandas as pd
import numpy as np


In [2]:
#import dataset
data = pd.read_csv('language_dataset.csv')

MINOR EDA   

In [3]:
#check for the shape of dataset and missing values
data.shape
data.isnull().sum()

Unnamed: 0    0
Text          0
language      0
dtype: int64

In [4]:
#drop unnecessary column
data.drop(columns='Unnamed: 0', inplace=True)
data.columns

Index(['Text', 'language'], dtype='object')

In [ ]:
#check for number of observations per language
data['language'].value_counts()

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


FEATURE ENGINEERING

In [7]:
#extract feature to token
vectorizer = CountVectorizer()


In [8]:
#separate X and Y
x_data = data[['Text']]
y_data = data[['language']]

In [9]:
#extract X features and check new shape
x_data_vectorized = vectorizer.fit_transform(x_data['Text'])
x_data_vectorized.shape

(22000, 277720)

In [10]:
#split X and Y data with balanced number of observations in target variable
x_train, x_test, y_train, y_test = train_test_split(x_data_vectorized, y_data, test_size= 0.3, random_state=34, stratify = y_data)

MULTINOMIAL LOGISTIC REGRESSION MODEL

In [62]:

#fit log model and check for accuracy
from sklearn.linear_model import LogisticRegression
multi_log = LogisticRegression(multi_class= 'multinomial', random_state=37)
multi_model = multi_log.fit(x_train, y_train['language'])
multi_model.score(x_test, y_test)

#94.6%

c:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


0.9466666666666667

In [65]:
#predict and check full metrics report
from sklearn.metrics import classification_report
multi_log_y_predicted = multi_model.predict(x_test)
multi_log_metrics = classification_report(y_test, multi_log_y_predicted)
print(multi_log_metrics)

              precision    recall  f1-score   support

      Arabic       1.00      0.97      0.99       300
     Chinese       0.80      0.51      0.63       300
       Dutch       1.00      0.98      0.99       300
     English       0.89      0.98      0.93       300
    Estonian       0.99      0.93      0.96       300
      French       1.00      1.00      1.00       300
       Hindi       1.00      0.97      0.98       300
  Indonesian       0.99      0.98      0.99       300
    Japanese       0.53      0.93      0.67       300
      Korean       1.00      0.93      0.97       300
       Latin       0.96      0.96      0.96       300
     Persian       1.00      0.97      0.98       300
   Portugese       0.98      0.98      0.98       300
      Pushto       1.00      0.96      0.98       300
    Romanian       1.00      0.98      0.99       300
     Russian       0.99      0.93      0.96       300
     Spanish       0.99      0.98      0.98       300
     Swedish       1.00    

RANDOM FOREST MODEL

In [22]:
#fit model and check accuracy
from sklearn.ensemble import RandomForestClassifier
RF_object = RandomForestClassifier(random_state=70)
RF_model = RF_object.fit(x_train, y_train['language'])
RF_model.score(x_test, y_test)

#92.1%


0.9216666666666666

In [ ]:
#predict and check full report
RF_y_predicted = RF_model.predict(x_test)
RF_metrics = classification_report(y_test, RF_y_predicted)
print(RF_metrics)

              precision    recall  f1-score   support

      Arabic       1.00      0.99      0.99       300
     Chinese       0.88      0.33      0.48       300
       Dutch       0.42      0.99      0.59       300
     English       0.82      0.99      0.89       300
    Estonian       0.99      0.96      0.97       300
      French       0.98      1.00      0.99       300
       Hindi       1.00      0.97      0.98       300
  Indonesian       0.99      0.98      0.99       300
    Japanese       0.96      0.37      0.53       300
      Korean       1.00      0.95      0.98       300
       Latin       0.99      0.95      0.97       300
     Persian       1.00      0.99      0.99       300
   Portugese       1.00      0.99      0.99       300
      Pushto       1.00      0.96      0.98       300
    Romanian       1.00      0.99      0.99       300
     Russian       0.99      0.95      0.97       300
     Spanish       0.98      0.99      0.99       300
     Swedish       0.99    

SUPPORT VECTOR CLASSIFIER MODEL

In [64]:
#fit model and check accuracy
from sklearn.svm import SVC
SVC_object = SVC(random_state=12)
SVC_model = SVC_object.fit(x_train, y_train['language'])
SVC_model.score(x_test, y_test)

#89.4%

0.894090909090909

In [66]:
#predict and check full report
SVC_y_predicted = SVC_model.predict(x_test)
SVC_metrics = classification_report(SVC_y_predicted, y_test)
print(SVC_metrics)

              precision    recall  f1-score   support

      Arabic       0.94      1.00      0.97       282
     Chinese       0.35      0.47      0.40       223
       Dutch       0.94      1.00      0.97       283
     English       0.95      0.82      0.88       345
    Estonian       0.89      0.99      0.94       269
      French       0.96      0.99      0.97       289
       Hindi       0.93      1.00      0.96       278
  Indonesian       0.95      1.00      0.97       285
    Japanese       0.90      0.45      0.60       594
      Korean       0.91      0.92      0.91       296
       Latin       0.88      0.96      0.92       277
     Persian       0.91      1.00      0.95       274
   Portugese       0.93      0.98      0.96       285
      Pushto       0.95      0.66      0.78       428
    Romanian       0.95      1.00      0.97       285
     Russian       0.80      0.99      0.89       243
     Spanish       0.94      0.99      0.96       287
     Swedish       0.97    

MULTINOMIAL NAIVE BAYES MODEL

In [ ]:
#fit model and check accuracy
from sklearn.naive_bayes import MultinomialNB
MNB_object = MultinomialNB()
MNB_model = MNB_object.fit(x_train, y_train['language'])
MNB_model.score(x_test, y_test)

#95.5%

0.9554545454545454

In [ ]:
#predict and check full report
MNB_y_predicted = MNB_model.predict(x_test)
MNB_metrics = classification_report(MNB_y_predicted, y_test)
print(MNB_metrics)

              precision    recall  f1-score   support

      Arabic       1.00      1.00      1.00       300
     Chinese       0.53      0.96      0.69       167
       Dutch       0.99      0.99      0.99       301
     English       1.00      0.68      0.81       444
    Estonian       0.97      0.99      0.98       293
      French       1.00      0.96      0.98       312
       Hindi       0.97      1.00      0.98       290
  Indonesian       0.99      0.99      0.99       300
    Japanese       0.86      0.72      0.78       360
      Korean       0.98      1.00      0.99       294
       Latin       0.91      1.00      0.95       273
     Persian       1.00      1.00      1.00       299
   Portugese       0.95      1.00      0.97       286
      Pushto       0.97      1.00      0.98       291
    Romanian       0.98      1.00      0.99       296
     Russian       0.99      0.99      0.99       299
     Spanish       0.99      0.98      0.99       302
     Swedish       1.00    

TESTING MODELS ON A NEW TEXT

In [ ]:

text = '나는 센 강을 따라 걸으며 파리의 아름다움을 감상한다. 도시의 불빛이 밤에 별처럼 반짝인다'
new_text = vectorizer.transform([text])

In [60]:
#multinomial naive bayes
MNB_model.predict(new_text)

array(['Korean'], dtype='<U10')

In [ ]:
#random forest
RF_model.predict(new_text)

array(['Korean'], dtype=object)

In [67]:
#multinomial logistic regression
multi_log.predict(new_text)

array(['Japanese'], dtype=object)

In [68]:
#support vector classifier
SVC_model.predict(new_text)

array(['Japanese'], dtype=object)